In [ ]:
import json
import numpy as np
import h5py
import tensorflow as tf
import os
import cPickle as pkl
#import ipdb
import numpy
import copy
import pprint
from Recognizer import MathFormulaRecognizer
import warnings
import sys
import math
import time
from util import *
from collections import OrderedDict

from data_iterator import dataIterator
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
home_path = os.getcwd()
checkpoint_path = os.path.join(home_path,'save')
max_iters = 100000
batch_size = 8
valid_batch_size = 2
finetune_encoder_after = -1
#Evaluation Checkpoint
nEvaImages = 300
EvaEach = 2500
SummaryEach = 1000
device = "/gpu:0"
batch_Imagesize=250000
valid_batch_Imagesize=250000
maxImagesize = 500000
maxlen = 200
n_epoch = 10000

In [ ]:
model = MathFormulaRecognizer(num_label =112,dim_hidden=128,device = '/device:GPU:0')
words,alphas= model.build_greedy_eval()
saver = tf.train.Saver(max_to_keep=10)
sess = tf.Session()
saved_path=tf.train.latest_checkpoint(checkpoint_path)
tf.reset_default_graph()
saver.restore(sess, saved_path)

In [ ]:
datasets=['./data/offline-train.pkl',
'./data/train_caption.txt']
valid_datasets=['./data/offline-test.pkl',
'./data/test_caption.txt']
dictionaries=['./data/dictionary.txt']


worddicts = load_dict(dictionaries[0])
worddicts_r = [None] * len(worddicts)

for kk, vv in worddicts.iteritems():
        worddicts_r[vv] = kk
train,train_uid_list = dataIterator(datasets[0], datasets[1],
                         worddicts,
                         batch_size=batch_size, batch_Imagesize=batch_Imagesize,maxlen=maxlen,maxImagesize=maxImagesize)
valid,valid_uid_list = dataIterator(valid_datasets[0], valid_datasets[1],
                     worddicts,
                     batch_size=valid_batch_size, batch_Imagesize=valid_batch_Imagesize,maxlen=maxlen,maxImagesize=maxImagesize)


In [ ]:
ind = 425
valid = np.squeeze(valid)
train = np.squeeze(train)
x, x_mask, y, y_mask = prepare_data(train[ind,0],train[ind,1])
x = x[0:1,:,:,:]
x_mask = x_mask[0:1,:,:]
Words,Alphas,height,width= sess.run([words,alphas,model.feature_height,model.feature_width],\
            feed_dict={model.x:x,model.x_mask:x_mask,model.is_train:False})

print(Words)

In [ ]:
im =np.squeeze(x[0])
plt.imshow(np.squeeze(im),cmap='gray')

In [ ]:
im =np.reshape(Alphas[11][0],[height,width])

plt.imshow(np.squeeze(im),cmap='gray')